In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# 分离出validation
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:70%]', 'train[70%:]', 'test'),
    as_supervised=True)


train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

train_labels_batch

embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

hub_layer(train_examples_batch[:3])


model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
tf.keras.layers.Dropout(0.2),
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()


model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=2)

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               2688      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 419,349
Trainable params: 419,349
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
35/35 - 3s - loss: 0.6663 - accuracy: 0.6003 - val_loss: 0.6466 - val_accuracy: 0.6623
Epoch 2/20
35/35 - 3s - loss: 0.6393 - accuracy: 0.6860 - val_loss: 0.6275 - val_accuracy: 0.7019
Epoch 3/20
35/35 - 2s - loss: 0.6205 - accuracy: 0